In [61]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf

mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)  #

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [63]:
n_train = mnist.train.num_examples  # 55,000
n_validation = mnist.validation.num_examples  # 5000
n_test = mnist.test.num_examples  # 10,000

n_input = 784  # input layer (28x28 pixels)
n_hidden1 = 512  # 1st hidden layer
n_hidden2 = 256  # 2nd hidden layer
n_hidden3 = 128  # 3rd hidden layer
n_output = 10  # output layer (0-9 digits)

learning_rate = 1e-4
n_iterations = 1000
batch_size = 128
dropout = 0.5

In [65]:
# Placeholders.
X = tf.placeholder("float", [None, n_input])
Y = tf.placeholder("float", [None, n_output])
keep_prob = tf.placeholder(tf.float32)

weights = {
    'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
    'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
    'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
    'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1)),
}

biases = {
    'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
    'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
    'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
    'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
}

layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
layer_drop = tf.nn.dropout(layer_3, rate = (1 - keep_prob)
output_layer = tf.matmul(layer_3, weights['out']) + biases['out']

W1007 11:18:00.947828 4533450176 deprecation.py:506] From <ipython-input-65-edfcf56db2f3>:23: calling dropout (from tensorflow.python.ops.nn_ops) with keep_prob is deprecated and will be removed in a future version.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [64]:
cross_entropy = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=output_layer ))
train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)

correct_pred = tf.equal(tf.argmax(output_layer, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

NameError: name 'output_layer' is not defined

In [ ]:
init = tf.global_variables_initializer()
sess = tf.Session()
sess.run(init)

# train on mini batches
for i in range(n_iterations):
    batch_x, batch_y = mnist.train.next_batch(batch_size)
    sess.run(train_step, feed_dict={
        X: batch_x, Y: batch_y, keep_prob: dropout
        })

    # print loss and accuracy (per minibatch)
    if i % 100 == 0:
        minibatch_loss, minibatch_accuracy = sess.run(
            [cross_entropy, accuracy],
            feed_dict={X: batch_x, Y: batch_y, keep_prob: 1.0}
            )
        print(
            "Iteration",
            str(i),
            "\t| Loss =",
            str(minibatch_loss),
            "\t| Accuracy =",
            str(minibatch_accuracy)
            )
        
test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: mnist.test.labels, keep_prob: 1.0})
print("\nAccuracy on test set:", test_accuracy)

In [ ]:


class Mach:

    def __init__(self):
        (self.x_train, self.y_train), (self.x_test, self.y_test) = tf.keras.datasets.mnist.load_data()
        self.batch_size = 5
        
        self.graph = tf.Graph()
        self.session = tf.Session(graph=self.graph)
        with self.graph.as_default():
            self.model_fn()
    
    def model_fn(self):
        
        
        learning_rate = 1e-4
        n_iterations = 1000
        batch_size = 128
        dropout = 0.5
        
        # Placeholders.
        self.image_batch = tf.compat.v1.placeholder(tf.float32, shape=[None, 28, 28], name="image")
        self.labels_batch = tf.compat.v1.placeholder(tf.float32, shape=[None, 1], name="labels")
        keep_prob = tf.placeholder(tf.float32)
        _input = tf.reshape(self.image_batch, [-1, 784])
        _labels = tf.one_hot(self.labels_batch, depth=tf.shape(self.labels_batch))

        
        weights = {
            'w1': tf.Variable(tf.truncated_normal([n_input, n_hidden1], stddev=0.1)),
            'w2': tf.Variable(tf.truncated_normal([n_hidden1, n_hidden2], stddev=0.1)),
            'w3': tf.Variable(tf.truncated_normal([n_hidden2, n_hidden3], stddev=0.1)),
            'out': tf.Variable(tf.truncated_normal([n_hidden3, n_output], stddev=0.1)),
        }
            
        biases = {
            'b1': tf.Variable(tf.constant(0.1, shape=[n_hidden1])),
            'b2': tf.Variable(tf.constant(0.1, shape=[n_hidden2])),
            'b3': tf.Variable(tf.constant(0.1, shape=[n_hidden3])),
            'out': tf.Variable(tf.constant(0.1, shape=[n_output]))
        }
        
        layer_1 = tf.add(tf.matmul(X, weights['w1']), biases['b1'])
        layer_2 = tf.add(tf.matmul(layer_1, weights['w2']), biases['b2'])
        layer_3 = tf.add(tf.matmul(layer_2, weights['w3']), biases['b3'])
        layer_drop = tf.nn.dropout(layer_3, keep_prob)
        output_layer = tf.matmul(layer_3, weights['out']) + biases['out']
        
         
        cross_entropy = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=output_layer ))
        train_step = tf.train.AdamOptimizer(1e-4).minimize(cross_entropy)
        
    def Train(self):
        
        index = 0
        while True:
            inputs = self.x_train[index:self.batch_size]
            labels = self.y_train[index:self.batch_size]
            feed_dict={self.image_batch: inputs, self.labels_batch:labels}
            fetches=[self.loss]
            run_output = self.session.run(fetches=fetches, feed_dict=feed_dict)
            #print (run_output[0])
            break
            




In [57]:
m = Mach()

ValueError: logits and labels must have the same shape ((?, 10) vs (?,))

In [58]:
m.Train()

ValueError: Cannot feed value of shape (1, 5) for Tensor 'labels:0', which has shape '(?, 10)'